In [ ]:
# Assignment 06 - Artificial Neural Networks
# CS 131 - Artificial Intelligence
# Darcy Corson (dcorso01)
# May 2024
#
# This Python script implements a neural network from scratch to classify Iris species based on their flower measurements.
# It leverages numpy for numerical operations, pandas for data handling, and scikit-learn for data preprocessing and splitting.
# The script features functions for both forward and backward propagation, loss calculations, and the training process includes
# early stopping for optimization. Users can interactively input flower measurements to receive predictions on Iris species.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# Assignment 06 - Artificial Neural Networks
# CS 131 - Artificial Intelligence
# Darcy Corson (dcorso01)
# May 2024
#
# This Python script implements a neural network from scratch to classify Iris species based on their flower measurements. 
# It leverages numpy for numerical operations, pandas for data handling, and scikit-learn for data preprocessing and splitting. 
# The script features functions for both forward and backward propagation, loss calculations, and the training process includes 
# early stopping for optimization. Users can interactively input flower measurements to receive predictions on Iris species.

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

def mse_loss(y_true, y_pred):
    """
    Purpose: calculates the mean squared error loss between the true labels and the predictions during training; helps in monitoring 
    the performance of the model during the training phase
    Inputs:
        y_true: the true labels
        y_pred: the predicted outputs
    Outputs: (float): computed MSE value
    Effects: none
    """
    return ((y_true - y_pred) ** 2).mean()

def sigmoid(x):
    """
    Purpose: the activation function for the hidden layer neurons, transforming the weighted inputs to a non-linear output
    Inputs: x: input array or scalar 
    Outputs: result of applying the sigmoid function element-wise
    Effects: none
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """
    Purpose: calculates the gradient of the sigmoid function for updating the weights
    Inputs: x: input array or scalar
    Outputs: derivative of the sigmoid function applied element-wise
    Effects: none
    """
    return x * (1 - x)

def softmax(x):
    """
    Purpose: converts logits to probabilities that sum to one
    Inputs: x: input array of logits
    Outputs: probabilities from applying softmax
    Effects: none
    """
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

def categorical_cross_entropy_loss(y_true, y_pred):
    """
    Purpose: computes the loss between the one-hot encoded true labels and the predicted probabilities from the softmax function
    Inputs:
        y_true: one-hot encoded true labels
        y_pred: predicted probabilities
    Outputs: calculated loss value
    Effects: none
    """
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = -np.sum(y_true * np.log(y_pred)) / len(y_true)
    return loss

class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        """
        Purpose: initialize the neural network 
        Inputs:
            input_nodes: Number of input nodes.
            hidden_nodes: Number of hidden nodes.
            output_nodes: Number of output nodes.
        Outputs: none
        Effects: sets initial weights and biases, configures the layer sizes, sets learning rate
        """
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights 
        self.weights_input_hidden = np.random.randn(self.input_nodes, self.hidden_nodes) * np.sqrt(2. / self.input_nodes)
        self.weights_hidden_output = np.random.randn(self.hidden_nodes, self.output_nodes) * np.sqrt(2. / self.hidden_nodes)

        # Initialize biases 
        self.bias_hidden = np.zeros((1, self.hidden_nodes))
        self.bias_output = np.zeros((1, self.output_nodes))

        self.lr = 0.01  # Learning rate

    def feedforward(self, X):
        """
        Purpose: performs a forward pass through the network
        Inputs:
            X: input features
        Outputs:
            softmax output of the network
        Effects:
            sets hidden and output layer activations as class attributes
        """
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output_layer_output = softmax(self.output_layer_input)
        return self.output_layer_output


    def backpropagation(self, inputs, expected_output, output):
        """
        Purpose: adjusts the weights and biases based on the error gradient
        Inputs:
            inputs: Input data for one training sample.
            expected_output: Expected output (target) for the corresponding input.
            output: Actual output produced by the feedforward pass before backpropagation.
        Outputs:
        Effects: updates the weights and biases between/for input-hidden and hidden-output layers
        """
        # Error in output layer 
        error_output_layer = expected_output - output
        
        # Error in hidden layer
        error_hidden_layer = error_output_layer.dot(self.weights_hidden_output.T)
        d_hidden_layer = error_hidden_layer * sigmoid_derivative(self.hidden_layer_output)
        
        # Update weights and biases
        self.weights_hidden_output += self.hidden_layer_output.T.dot(error_output_layer) * self.lr
        self.weights_input_hidden += inputs.T.dot(d_hidden_layer) * self.lr
        self.bias_hidden += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.lr
        self.bias_output += np.sum(error_output_layer, axis=0, keepdims=True) * self.lr

    def train(self, X_train, y_train, X_val, y_val, epochs, patience=100):
        """
        Purpose: manages the iterative training process
        Inputs:
            X_train: training dataset inputs
            y_train: training dataset targets
            X_val: validation dataset inputs
            y_val: validation dataset targets
            epochs: total number of epochs to train
            patience: number of epochs to wait for improvement in validation loss before early stopping
        Outputs: none
        Effects: trains the neural network by adjusting weights and biases
        """
        best_val_loss = np.inf
        patience_counter = 0  

        for epoch in range(epochs):
            output_train = []
            for inputs, labels in zip(X_train, y_train):
                inputs = np.array([inputs])  
                outputs = self.feedforward(inputs)
                output_train.append(outputs)
                self.backpropagation(inputs, labels, outputs)

            # Calculate training MSE and accuracy
            output_train = np.vstack(output_train)  # Stack output arrays
            train_mse = mse_loss(y_train, output_train)
            predicted_classes = np.argmax(output_train, axis=1)
            true_classes = np.argmax(y_train, axis=1)
            train_accuracy = np.mean(predicted_classes == true_classes) * 100
            
            # Calculate validation loss
            val_outputs = self.feedforward(X_val)
            val_loss = categorical_cross_entropy_loss(y_val, val_outputs)
            print(f'Epoch {epoch+1}, Training MSE = {train_mse:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}')
            
            # Early stopping logic
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0  
            else:
                patience_counter += 1  
            
            if patience_counter >= patience:
                print(f'Stopping early after {epoch+1} epochs')
                break

def load_data(filepath):
    """
    Purpose: loads and preprocesses the dataset
    Inputs: filepath: path to the dataset file
    Outputs: a tuple containing ormalized feature values and one-hot encoded target labels
    Effects: normalizes feature columns and one-hot encodes the target labels
    """
    dataset = pd.read_csv(filepath, header=None)
    
    # Normalize features
    normalized_features = (dataset.iloc[:, :-1] - dataset.iloc[:, :-1].mean(axis=0)) / dataset.iloc[:, :-1].std(axis=0)
    
    # One-hot encode target labels
    encoder = LabelEncoder()
    targets = encoder.fit_transform(dataset.iloc[:, -1])
    targets = np.eye(np.unique(targets).size)[targets]  
    
    return normalized_features.values, targets

def predict(model, input_features):
    """
    Purpose: makes predictions on new data inputs using the trained network
    Inputs:
        model: trained neural network model
        input_features: features of the new data input for prediction
    Outputs: returns the predicted class label for the input data
    Effects: none
    """
    output = model.feedforward(input_features)
    class_labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    return class_labels[np.argmax(output)]

def evaluate_accuracy(test_outputs, y_test):
    """
    Purpose: assesses the performance of the network on test data
    Inputs:
        test_outputs: uutput predictions of the neural network for the test data
        y_test: actual target labels for the test data
    Outputs: the accuracy of the predictions
    Effects: none
    """
    
    test_outputs = np.squeeze(test_outputs, axis=1)
    encoder = LabelEncoder()
    true_classes = encoder.fit_transform(np.argmax(y_test, axis=1))
    predicted_classes = np.argmax(test_outputs, axis=1)
    true_classes_encoded = np.eye(np.unique(predicted_classes).size)[true_classes]
    
    # Calculate accuracy
    accuracy = np.mean(predicted_classes == np.argmax(true_classes_encoded, axis=1))
    print("Test accuracy:", accuracy)

    return accuracy

if __name__ == "__main__":
    filepath = '/Users/dcorson/Desktop/ANN - iris data.txt'
    X, y = load_data(filepath)
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)
    
    nn = NeuralNetwork(input_nodes=4, hidden_nodes=5, output_nodes=3)
    nn.train(X_train, y_train, X_val, y_val, epochs=1000)
    
    test_outputs = np.array([nn.feedforward(np.array([x])) for x in X_test])
    test_accuracy = evaluate_accuracy(test_outputs, y_test)
    print(f'Test Accuracy: {test_accuracy:.2f}')

    while True:
        print("\nEnter the sepal length, sepal width, petal length, and petal width of the Iris (or 'Q' to quit):")
        sl_input = input("Sepal length (cm): ")
        if sl_input.lower() == 'q':
            break

        try:
            sl = float(sl_input)
            sw = float(input("Sepal width (cm): "))
            pl = float(input("Petal length (cm): "))
            pw = float(input("Petal width (cm): "))
        except ValueError:
            print("Invalid input. Please enter numeric values.")
            continue

        user_input = np.array([[sl, sw, pl, pw]])
        dataset = pd.read_csv(filepath, header=None)  
        mean = dataset.iloc[:, :-1].mean(axis=0)
        std = dataset.iloc[:, :-1].std(axis=0)
        normalized_user_input = (user_input - mean.values) / std.values

        normalized_user_input = normalized_user_input.reshape(1, -1)

        prediction = predict(nn, normalized_user_input)
        print(f"The predicted Iris class is: {prediction}")
